# Exercise Set 7: Data structuring  3

*Morning, August 15, Morning*

In this Exercise Set we finalize our work with the weather data we started working on in Exercise Set 4. We will also study a dataset of traffic data from Copenhagen to iterate through the pandas workflow once more. 

In [1]:
%matplotlib inline 

import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl

# Increases the plot size a little
mpl.rcParams['figure.figsize'] = 11, 6


## Exercise Section 7.1: Weather data, part 3
We continue with the final part of three exercises on structuring weather data. In this exercise you must use the function for fetching and structuring weather data which you made in Exercise 6.1.5.

> **Ex. 7.1.1:** Plot the monthly max,min, mean, first and third quartiles for maximum temperature for our station with the ID _'ITE00100550'_ in 1864. 

> *Hint*: the method `describe` computes all these measures.

In [2]:
def get_weather_data(year):
    base_url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/{}.csv.gz'
    df_weather = pd.read_csv(base_url.format(str(year)),
                            compression='gzip',
                            header=None).iloc[:, :4] # Truncate rows to only the first 4 columns
    # name the columns
    df_weather.columns = ['station', 'datetime', 'obs_type', 'tmax_c']
    # Wrangle tempeture, it comes in tenth of degrees
    df_weather['tmax_c'] = df_weather['tmax_c'] / 10
    # Choose only the observation type TMAX
    df_select_tmax = df_weather[(df_weather.obs_type == 'TMAX')].copy()
    
    df_select_tmax['tmax_f'] = 32 + 1.8 * df_select_tmax['tmax_c']
    df_sorted = df_select_tmax.reset_index(drop=True).sort_values(by=['tmax_c'])
    return df_sorted

In [3]:
# [Answer to Ex. 7.1.1]

w1864 = get_weather_data(1864)


In [4]:
# [Answer to Ex. 4.1.3]
from pandas import DataFrame

def get_station_measurements(df: DataFrame, station_id: str) -> DataFrame:
    df_station = df.loc[(df['station'] == station_id)].copy()
    return df_station

w1864_ite = get_station_measurements(w1864, 'ITE00100550')
w1864_ite.head()
w1864_ite.tail()

,station,datetime,obs_type,tmax_c,tmax_f
3279,ITE00100550,18640801,TMAX,33.5,92.30
3342,ITE00100550,18640805,TMAX,33.5,92.30
3389,ITE00100550,18640808,TMAX,34.1,93.38
3294,ITE00100550,18640802,TMAX,34.6,94.28
3405,ITE00100550,18640809,TMAX,34.8,94.64


In [5]:
from datetime import datetime

def get_month(int_format: int) -> int:
    d = pd.to_datetime(int_format, format='%Y%m%d')
    return d.month

w1864_ite['month'] = w1864_ite['datetime'].apply(get_month)

w1864_ite.head()

,station,datetime,obs_type,tmax_c,tmax_f,month
237,ITE00100550,18640117,TMAX,-6.3,20.66,1
192,ITE00100550,18640114,TMAX,-5.0,23.00,1
251,ITE00100550,18640118,TMAX,-5.0,23.00,1
177,ITE00100550,18640113,TMAX,-4.3,24.26,1
207,ITE00100550,18640115,TMAX,-3.1,26.42,1


In [6]:
split_var = 'month'
apply_vars = ['tmax_c', 'tmax_f']

w1864_ite.groupby('month') \
    [apply_vars] \
    .describe()

tmax_c                                                         tmax_f  \
       count       mean       std   min     25%    50%     75%   max  count   
month                                                                         
1       31.0  -0.077419  3.064605  -6.3  -2.200   0.00   1.900   5.3   31.0   
2       29.0   4.134483  3.031652  -1.8   1.500   4.40   6.800   8.4   29.0   
3       31.0  12.200000  2.780408   7.9  10.050  12.10  14.150  18.0   31.0   
4       30.0  16.243333  4.487199   6.3  14.425  16.25  18.400  25.1   30.0   
5       31.0  21.470968  3.312622  14.0  19.500  21.10  23.800  27.0   31.0   
6       30.0  26.670000  2.926355  20.4  25.100  27.40  28.500  31.3   30.0   
7       31.0  29.387097  1.913068  25.4  28.450  29.80  30.900  32.5   31.0   
8       31.0  28.922581  3.794708  21.0  26.450  29.30  31.600  34.8   31.0   
9       30.0  24.103333  3.620772  18.0  21.500  23.35  26.800  29.9   30.0   
10      31.0  16.519355  2.375629  11.9  15.000  16.00  18.350  20.5   31.0   
11      30.0   9.703333  2.612633   5.0   7.675  10.20  11.375  14.6   30.0   
12      31.0   5.316129  2.175791   1.5   3.600   5.60   7.000   9.3   31.0   

                                                                 
            mean       std    min     25%    50%     75%    max  
month                                                            
1      31.860645  5.516290  20.66  28.040  32.00  35.420  41.54  
2      39.442069  5.456973  28.76  34.700  39.92  44.240  47.12  
3      53.960000  5.004734  46.22  50.090  53.78  57.470  64.40  
4      61.238000  8.076958  43.34  57.965  61.25  65.120  77.18  
5      70.647742  5.962719  57.20  67.100  69.98  74.840  80.60  
6      80.006000  5.267438  68.72  77.180  81.32  83.300  88.34  
7      84.896774  3.443522  77.72  83.210  85.64  87.620  90.50  
8      84.060645  6.830474  69.80  79.610  84.74  88.880  94.64  
9      75.386000  6.517389  64.40  70.700  74.03  80.240  85.82  
10     61.734839  4.276132  53.42  59.000  60.80  65.030  68.90  
11     49.466000  4.702739  41.00  45.815  50.36  52.475  58.28  
12     41.569032  3.916423  34.70  38.480  42.08  44.600  48.74

> **Ex. 7.1.2:** Get the processed data from years 1864-1867 as a list of DataFrames. Convert the list into a single DataFrame by concatenating vertically. 

In [7]:
# [Answer to Ex. 7.1.2]

years = list(range(1864, 1868))
four_year_dfs = {year:get_weather_data(year) for year in years} # dictionary of all stock price
weather_df = pd.concat(four_year_dfs, axis=0) # vertical axis
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], format='%Y%m%d')
# stock_df.plot(logy=True, figsize=(10,3))
weather_df.reset_index()
# four_year_df = [dfs_list.append(get_weather_data(year)) for year in years]

,level_0,level_1,station,datetime,obs_type,tmax_c,tmax_f
0,1864,845,SZ000006717,1864-02-26,TMAX,-34.0,-29.20
1,1864,577,SZ000006717,1864-02-08,TMAX,-17.4,0.68
2,1864,699,CA006158350,1864-02-17,TMAX,-16.7,1.94
3,1864,42,SZ000006717,1864-01-03,TMAX,-16.5,2.30
4,1864,16,CA006158350,1864-01-02,TMAX,-16.1,3.02
5,1864,607,SZ000006717,1864-02-10,TMAX,-15.9,3.38
6,1864,199,EZE00100082,1864-01-14,TMAX,-15.1,4.82
7,1864,592,SZ000006717,1864-02-09,TMAX,-15.0,5.00
8,1864,741,SZ000006717,1864-02-19,TMAX,-14.7,5.54
9,1864,533,SZ000006717,1864-02-05,TMAX,-14.4,6.08


In [8]:
# Sanity check
weather_df[weather_df['datetime'] == '18640226'].head()

station   datetime obs_type  tmax_c  tmax_f
1864 845  SZ000006717 1864-02-26     TMAX   -34.0  -29.20
     833  CA006158350 1864-02-26     TMAX    -1.1   30.02
     837  GME00125218 1864-02-26     TMAX     2.1   35.78
     842  GM000004204 1864-02-26     TMAX     2.5   36.50
     835  BE000006447 1864-02-26     TMAX     3.8   38.84

In [10]:
weather_df.head()

station   datetime obs_type  tmax_c  tmax_f
1864 845  SZ000006717 1864-02-26     TMAX   -34.0  -29.20
     577  SZ000006717 1864-02-08     TMAX   -17.4    0.68
     699  CA006158350 1864-02-17     TMAX   -16.7    1.94
     42   SZ000006717 1864-01-03     TMAX   -16.5    2.30
     16   CA006158350 1864-01-02     TMAX   -16.1    3.02

> **Ex. 7.1.3:** Parse the station location data which you can find at https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt. Merge station locations onto the weather data spanning 1864-1867.  

> _Hint:_ The location data have the folllowing format, 

```
------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------
```

> *Hint*: The station information has fixed width format - does there exist a pandas reader for that?

In [11]:
# [Answer to Ex. 7.1.3]
url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'

col_specs = [(0, 10), (12, 19), (21, 29), (31, 36), (38, 39), (41, 70), (72, 74), (76, 78), (80, 84)]
# col_spec = [ t for non_zero_based]

station_geo_data = pd.read_fwf(url,
                   header=None,
                   colspecs=col_specs)

In [12]:
colnames = ['id', 'latitude', 'longitude', 'elevation', 'state', 'name', 'gsn_flag', 'hcn/crn_flag', 'wmo_id']
station_geo_data.columns = colnames
station_geo_data.shape

(113933, 9)

In [13]:
station_geo_data.head()

,id,latitude,longitude,elevation,state,name,gsn_flag,hcn/crn_flag,wmo_id
0,ACW0001160,17.116,-61.783,10.0,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW0001164,17.133,-61.783,19.0,NaN,ST JOHNS,NaN,NaN,NaN
2,AE00004119,25.333,55.517,34.0,NaN,SHARJAH INTER. AIRP,GS,NaN,4119.0
3,AEM0004119,25.255,55.364,10.0,NaN,DUBAI INTL,NaN,NaN,4119.0
4,AEM0004121,24.433,54.651,26.0,NaN,ABU DHABI INTL,NaN,NaN,4121.0


In [14]:
station_geo_data.query('latitude < 0').head()

,id,latitude,longitude,elevation,state,name,gsn_flag,hcn/crn_flag,wmo_id
219,AO00006616,-8.850,13.233,74.0,NaN,LUANDA,GS,NaN,6616.0
220,AO00006627,-11.417,15.117,1304.0,NaN,WAKU KUNGU (CELA),GS,NaN,6627.0
221,AO00006639,-14.933,13.567,1761.0,NaN,LUBANGO (SA DA BAND,GS,NaN,6639.0
222,AO00006641,-14.650,17.683,1343.0,NaN,MENONGUE (SERPA PIN,GS,NaN,6641.0
223,AO00006642,-15.200,12.150,45.0,NaN,MOCAMEDES,GS,NaN,6642.0


## Exercise Section 7.2: Traffic data in Copenhagen

In this second part of exercise set 7 you will be working with traffic data from Copenhagen Municipality.

The municipality have made the data openly available through the [opendata.dk](http://www.opendata.dk/) platform. We will use the data from traffic counters to construct a dataset of hourly traffic. We will use this data to get basic insights on the development in traffic over time and relate it to weather. The gist here is to practice a very important skill in Data Science: being able to quickly fetch data from the web and structure it so that you can work with it. Scraping usually gets a bit more advanced than what we will do today, but the following exercises should give you a taste for how it works. The bulk of these exercise, however, revolve around using the Pandas library to structure and analyze data.

### Part 7.2.a: getting some data to work with

Hence follows a simple scraping exercise where you (1) collect urls for datasets in the webpage listing data on traffic counters and (2) use these urls to load the data into one dataframe.

> **Ex. 7.2.1:** Using the requests module, extract the html markup of the webpage data.kk.dk/dataset/faste-trafiktaellinger and store it as a string in a new variable.

In [15]:
# [Answer to Ex. 7.2.1]
import requests

traffic_count_url = 'https://data.kk.dk/dataset/faste-trafiktaellinger'

response = requests.get(traffic_count_url)

print(response.text[:1000])

<!DOCTYPE html>
<!--[if IE 7]> <html lang="da_DK" class="ie ie7"> <![endif]-->
<!--[if IE 8]> <html lang="da_DK" class="ie ie8"> <![endif]-->
<!--[if IE 9]> <html lang="da_DK" class="ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="da_DK"> <!--<![endif]-->
  <head>
    <!--[if lte ie 8]><script type="text/javascript" src="/fanstatic/vendor/:version:2017-02-24T11:53:25/html5.min.js"></script><![endif]-->
<link rel="stylesheet" type="text/css" href="/fanstatic/vendor/:version:2017-02-24T11:53:25/select2/select2.css" />
<link rel="stylesheet" type="text/css" href="/fanstatic/css/:version:2017-02-24T11:53:25/main.min.css" />
<link rel="stylesheet" type="text/css" href="/fanstatic/vendor/:version:2017-02-24T11:53:25/font-awesome/css/font-awesome.min.css" />
<!--[if ie 7]><link rel="stylesheet" type="text/css" href="/fanstatic/vendor/:version:2017-02-24T11:53:25/font-awesome/css/font-awesome-ie7.min.css" /><![endif]-->
<link rel="stylesheet" type="text/css" href="/fanstatic/ckanext-geov

> **Ex. 7.2.2:** Using the re module, extract a list of all the urls in the html string and store them in a new variable.

> _Hint:_ Try using the re.findall method. You may want to Google around to figure out how to do this. Protip: searching for something along the lines of "extract all links in html regex python" and hitting the first StackOverflow link will probably get you farther than reading elaborate documentation.

In [16]:
# [Answer to Ex. 7.2.2]
import re

def get_all_hrefs(html_text):
    return re.findall('(?P<url>https?://[^\s]+)"', html_text)

all_links = get_all_hrefs(response.text)

print(all_links)

['https://data-kk.innocraft.cloud/', 'https://data.kk.dk/', 'https://data.kk.dk/', 'http://www.w3.org/1999/xhtml', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57/download/faste-trafiktaellinger-2005.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/0bfc0425-0b2f-43b4-be71-117b4265598d/download/faste-trafiktaellinger-2006.xlsx', 'http://www.w3.org/1999/xhtml', 'http://data.kk.dk/storage/f/2015-05-11T07%3A35%3A06.512Z/faste-trafiktaellinger-2007.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/e4f321ad-76bc-483f-839e-e04cfea6bfc0/download/faste-trafiktaellinger-2008.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/bac5c582-bdd0-45db-b174-f835e82851bc/download/faste-trafiktaellinger-2009.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/a209aef9-802c-4cda-9b42-f4364877b04f/download/faste-trafiktaelling

> **Ex. 7.2.3:** Create a new variable that only contains the links that point to downloadable traffic data sheets. 

> _Hint:_ You want to filter the results from above. For example to only include urls with the term 'download' in them.

In [17]:
# [Answer to Ex. 7.2.3]

# determines if a string ends with .xlsx
def is_xlsx_file(filename: str) -> bool:
    return filename.endswith('.xlsx')

# filters a list of strings using the above is_xlsx_file function
def get_all_xlsx_links(urls):
    return list(filter(is_xlsx_file, urls))

all_xlsx_links = get_all_xlsx_links(all_links)

print(all_xlsx_links)

['https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57/download/faste-trafiktaellinger-2005.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/0bfc0425-0b2f-43b4-be71-117b4265598d/download/faste-trafiktaellinger-2006.xlsx', 'http://data.kk.dk/storage/f/2015-05-11T07%3A35%3A06.512Z/faste-trafiktaellinger-2007.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/e4f321ad-76bc-483f-839e-e04cfea6bfc0/download/faste-trafiktaellinger-2008.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/bac5c582-bdd0-45db-b174-f835e82851bc/download/faste-trafiktaellinger-2009.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/a209aef9-802c-4cda-9b42-f4364877b04f/download/faste-trafiktaellinger-2010.xlsx', 'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/8b2f60a9-17ac-437d-ae6e-ad44c0a49edd/download/faste-trafi

> **Ex. 7.2.4:** Using pd.read_excel method, load the datasets into a list. Your resulting variable should hold a list of Pandas dataframes.

> _Hint:_ you may want to set the skiprows= keyword argument. 

In [19]:
all_xlsx_links[0]

'https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57/download/faste-trafiktaellinger-2005.xlsx'

In [22]:
test = pd.read_excel(all_xlsx_links[0])


In [23]:
test.head()

,Mastra,Unnamed: 1,Unnamed: 2,Talt døgn/time matrice,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Udskr.,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gruppe,NaN,JOSBO_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Periode,NaN,01.01.2005-31.12.2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Køretøjsart,NaN,MOTORKTJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
test2 = pd.read_excel(all_xlsx_links[0], skiprows=10, header=0)

In [59]:
test2.head()

,Vej-Id,Vejnavn,Spor,(UTM32),(UTM32).1,Dato,kl.00-01,kl.01-02,kl.02-03,kl.03-04,...,kl.14-15,kl.15-16,kl.16-17,kl.17-18,kl.18-19,kl.19-20,kl.20-21,kl.21-22,kl.22-23,kl.23-24
0,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,01.04.2005,83,52.0,86,133,...,955.0,954.0,805.0,759.0,537.0,424.0,358.0,349.0,312.0,219.0
1,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,02.04.2005,153,135.0,95,123,...,748.0,657.0,745.0,618.0,466.0,363.0,335.0,318.0,307.0,263.0
2,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,03.04.2005,182,161.0,113,120,...,682.0,691.0,647.0,604.0,546.0,455.0,384.0,304.0,192.0,92.0
3,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,04.04.2005,57,61.0,70,125,...,804.0,947.0,839.0,702.0,513.0,410.0,372.0,340.0,194.0,97.0
4,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,05.04.2005,73,52.0,61,120,...,827.0,988.0,847.0,806.0,592.0,450.0,349.0,353.0,196.0,91.0


In [40]:
# [Answer to Ex. 7.2.4]

# log wrapper for pd.read_excel(args)
def log_read_url(url):
    print('Downloading: {}'.format(url))
    return pd.read_excel(url, skiprows=10)

traffic_list = [log_read_url(url) for url in all_xlsx_links]

Downloading: https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/dcb49b51-1a17-40f6-82b4-34eaf58bce57/download/faste-trafiktaellinger-2005.xlsx
Downloading: https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/0bfc0425-0b2f-43b4-be71-117b4265598d/download/faste-trafiktaellinger-2006.xlsx
Downloading: http://data.kk.dk/storage/f/2015-05-11T07%3A35%3A06.512Z/faste-trafiktaellinger-2007.xlsx
Downloading: https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/e4f321ad-76bc-483f-839e-e04cfea6bfc0/download/faste-trafiktaellinger-2008.xlsx
Downloading: https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/bac5c582-bdd0-45db-b174-f835e82851bc/download/faste-trafiktaellinger-2009.xlsx
Downloading: https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160bc61/resource/a209aef9-802c-4cda-9b42-f4364877b04f/download/faste-trafiktaellinger-2010.xlsx
Downloading: https://data.kk.dk/dataset/5f9345b0-d73b-43ff-a659-e468d160

In [41]:
print(traffic_list)

[                       Vej-Id       Vejnavn  Spor  (UTM32)  (UTM32).1  \
0      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
1      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
2      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
3      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
4      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
5      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
6      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
7      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
8      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
9      101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
10     101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
11     101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777   
12     101 1011492-0 1/ 124 -  Ellebjergvej   NaN 

In [42]:
for df in traffic_list:
    print(df.head())
    print()

                   Vej-Id       Vejnavn  Spor  (UTM32)  (UTM32).1        Dato  \
0  101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777  01.04.2005   
1  101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777  02.04.2005   
2  101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777  03.04.2005   
3  101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777  04.04.2005   
4  101 1011492-0 1/ 124 -  Ellebjergvej   NaN   720881    6172777  05.04.2005   

   kl.00-01  kl.01-02  kl.02-03  kl.03-04  ...  kl.14-15  kl.15-16  kl.16-17  \
0        83      52.0        86       133  ...     955.0     954.0     805.0   
1       153     135.0        95       123  ...     748.0     657.0     745.0   
2       182     161.0       113       120  ...     682.0     691.0     647.0   
3        57      61.0        70       125  ...     804.0     947.0     839.0   
4        73      52.0        61       120  ...     827.0     988.0     847.0   

   kl.17-18  kl.18-19  kl.19-20 

In [ ]:
traffic_list[0].head()

> **Ex. 7.2.5:** Merge the list of dataframes into a single dataframe.

In [108]:
# [Answer to Ex. 7.2.5]
# test_df = pd.merge(traffic_list[0], traffic_list[1])
all_traffic_df = pd.concat(traffic_list, join='outer', axis=0, sort=False)
# test_df = pd.concat([traffic_list[0], traffic_list[1]], join='outer', axis=0, sort=False)

In [109]:
all_traffic_df.shape

(183397, 30)

In [110]:
all_traffic_df.tail()

,Vej-Id,Vejnavn,Spor,(UTM32),(UTM32).1,Dato,kl.00-01,kl.01-02,kl.02-03,kl.03-04,...,kl.14-15,kl.15-16,kl.16-17,kl.17-18,kl.18-19,kl.19-20,kl.20-21,kl.21-22,kl.22-23,kl.23-24
13706,101 1018541-0 3/ 638 T,Ørestads Boulevard,NaN,725458,6171542,27.12.2014,51.0,34.0,11.0,19.0,...,993.0,932.0,887.0,789.0,592.0,446.0,338.0,265.0,130.0,68.0
13707,101 1018541-0 3/ 638 T,Ørestads Boulevard,NaN,725458,6171542,28.12.2014,64.0,36.0,23.0,19.0,...,950.0,907.0,707.0,674.0,539.0,401.0,294.0,235.0,121.0,62.0
13708,101 1018541-0 3/ 638 T,Ørestads Boulevard,NaN,725458,6171542,29.12.2014,40.0,23.0,36.0,20.0,...,993.0,977.0,1036.0,794.0,677.0,584.0,420.0,345.0,124.0,66.0
13709,101 1018541-0 3/ 638 T,Ørestads Boulevard,NaN,725458,6171542,30.12.2014,65.0,36.0,23.0,17.0,...,1184.0,1169.0,1227.0,1039.0,869.0,817.0,657.0,502.0,245.0,110.0
13710,101 1018541-0 3/ 638 T,Ørestads Boulevard,NaN,725458,6171542,31.12.2014,66.0,30.0,11.0,21.0,...,760.0,464.0,291.0,234.0,86.0,93.0,68.0,83.0,81.0,76.0


### Part 7.2.b Structuring your data

If you successfully completed the previous part, you should now have a dataframe with about 183.397 rows (if your number of rows is close but not the same, worry not—it matters little in the following). Well done! But the data is still in no shape for analysis, so we must clean it up a little.

161.236 rows (and 30 columns) is a lot of data. ~3.3 MB by my back-of-the-envelope calculations, so not "Big Data", but still enough to make your CPU heat up if you don't use it carefully. Pandas is built to handle fairly large dataframes and has advanced functionality to perform very fast operations even when the size of your data grows huge. So instead of working with basic Python we recommend working pandas built-in procedures as they are constructed to be fast on dataframes.

Nerd fact: the reason pandas is much faster than pure Python is that dataframes access a lower level programming languages (namely C, C++) which are multiple times faster than Python. The reason it is faster is that it has a higher level of explicitness and thus is more difficult to learn and navigate.

> **Ex. 7.2.6:** Reset the row indices of your dataframe so the first index is 0 and the last is whatever the number of rows your dataframe has. Also drop the column named 'index' and the one named `Spor`.

In [111]:
# [Answer to Ex. 7.2.6]
all_traffic_df.reset_index(drop=True) # drop=True makes sure that the index var is not saved

,Vej-Id,Vejnavn,Spor,(UTM32),(UTM32).1,Dato,kl.00-01,kl.01-02,kl.02-03,kl.03-04,...,kl.14-15,kl.15-16,kl.16-17,kl.17-18,kl.18-19,kl.19-20,kl.20-21,kl.21-22,kl.22-23,kl.23-24
0,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,01.04.2005,83.0,52.0,86.0,133.0,...,955.0,954.0,805.0,759.0,537.0,424.0,358.0,349.0,312.0,219.0
1,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,02.04.2005,153.0,135.0,95.0,123.0,...,748.0,657.0,745.0,618.0,466.0,363.0,335.0,318.0,307.0,263.0
2,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,03.04.2005,182.0,161.0,113.0,120.0,...,682.0,691.0,647.0,604.0,546.0,455.0,384.0,304.0,192.0,92.0
3,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,04.04.2005,57.0,61.0,70.0,125.0,...,804.0,947.0,839.0,702.0,513.0,410.0,372.0,340.0,194.0,97.0
4,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,05.04.2005,73.0,52.0,61.0,120.0,...,827.0,988.0,847.0,806.0,592.0,450.0,349.0,353.0,196.0,91.0
5,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,06.04.2005,75.0,49.0,80.0,143.0,...,855.0,968.0,855.0,708.0,511.0,523.0,371.0,362.0,212.0,115.0
6,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,07.04.2005,68.0,60.0,82.0,132.0,...,860.0,1000.0,808.0,745.0,519.0,425.0,405.0,341.0,234.0,120.0
7,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,08.04.2005,86.0,58.0,84.0,144.0,...,949.0,960.0,859.0,765.0,528.0,388.0,385.0,340.0,281.0,227.0
8,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,09.04.2005,142.0,124.0,115.0,115.0,...,708.0,661.0,656.0,584.0,410.0,401.0,357.0,327.0,325.0,257.0
9,101 1011492-0 1/ 124 -,Ellebjergvej,NaN,720881,6172777,10.04.2005,195.0,143.0,125.0,125.0,...,707.0,642.0,573.0,519.0,478.0,437.0,382.0,304.0,194.0,97.0


In [112]:
# Drop the 'Spor' column from the dataframe
all_traffic_df.drop('Spor', axis=1, inplace=True)

In [113]:
all_traffic_df.head()

,Vej-Id,Vejnavn,(UTM32),(UTM32).1,Dato,kl.00-01,kl.01-02,kl.02-03,kl.03-04,kl.04-05,...,kl.14-15,kl.15-16,kl.16-17,kl.17-18,kl.18-19,kl.19-20,kl.20-21,kl.21-22,kl.22-23,kl.23-24
0,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,01.04.2005,83.0,52.0,86.0,133.0,314.0,...,955.0,954.0,805.0,759.0,537.0,424.0,358.0,349.0,312.0,219.0
1,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,02.04.2005,153.0,135.0,95.0,123.0,145.0,...,748.0,657.0,745.0,618.0,466.0,363.0,335.0,318.0,307.0,263.0
2,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,03.04.2005,182.0,161.0,113.0,120.0,164.0,...,682.0,691.0,647.0,604.0,546.0,455.0,384.0,304.0,192.0,92.0
3,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,04.04.2005,57.0,61.0,70.0,125.0,312.0,...,804.0,947.0,839.0,702.0,513.0,410.0,372.0,340.0,194.0,97.0
4,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,05.04.2005,73.0,52.0,61.0,120.0,311.0,...,827.0,988.0,847.0,806.0,592.0,450.0,349.0,353.0,196.0,91.0


> **Ex. 7.2.7:** Rename variables from Danish to English using the dictionary below.

```python 
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}
```

In [118]:
# [Answer to Ex. 7.2.7]
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}

all_traffic_df.rename(columns = dk_to_uk, inplace=True)

In [119]:
all_traffic_df.head()

,road_id,road_name,UTM32_north,UTM32_east,date,kl.00-01,kl.01-02,kl.02-03,kl.03-04,kl.04-05,...,kl.14-15,kl.15-16,kl.16-17,kl.17-18,kl.18-19,kl.19-20,kl.20-21,kl.21-22,kl.22-23,kl.23-24
0,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,01.04.2005,83.0,52.0,86.0,133.0,314.0,...,955.0,954.0,805.0,759.0,537.0,424.0,358.0,349.0,312.0,219.0
1,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,02.04.2005,153.0,135.0,95.0,123.0,145.0,...,748.0,657.0,745.0,618.0,466.0,363.0,335.0,318.0,307.0,263.0
2,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,03.04.2005,182.0,161.0,113.0,120.0,164.0,...,682.0,691.0,647.0,604.0,546.0,455.0,384.0,304.0,192.0,92.0
3,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,04.04.2005,57.0,61.0,70.0,125.0,312.0,...,804.0,947.0,839.0,702.0,513.0,410.0,372.0,340.0,194.0,97.0
4,101 1011492-0 1/ 124 -,Ellebjergvej,720881,6172777,05.04.2005,73.0,52.0,61.0,120.0,311.0,...,827.0,988.0,847.0,806.0,592.0,450.0,349.0,353.0,196.0,91.0


Python is quite efficient. For example, when you create a new dataframe by manipulating an old one, Python notices that—apart from some minor changes—these two objects are almost the same. Since memory is a precious resource, Python will represent the values in the new dataframe as references to the variables in the old dataset. This is great for performance, but if you for whatever reason change some of the values in your old dataframe, values in the new one will also change—and we don't want that! Luckily, we can break this dependency.

> **Ex. 7.2.8:** Break the dependencies of the dataframe that resulted from Ex. 7.2.7 using the `.copy` method. Delete all other dataframes.

In [120]:
# [Answer to Ex. 7.2.8]
# As I choose to use inplace=True in all command, we actually don't have any other 'waste' data frames lying around

Once you have structured appropriately, something that you will want to do again and again is selecting subsets of the data. Specifically, it means that you select specific rows in the dataset based on some column values.

>**Ex. 7.2.9:** Create a new column in the dataframe called total that is True when the last letter of road_id is T and otherwise False.

> _Hint:_ you will need the `pd.Series.str` attribute for this.

In [ ]:
# [Answer to Ex. 7.2.9]


> **Ex. 7.2.10:** Select rows where total is True. Delete all the remaining observations.

In [ ]:
# [Answer to Ex. 7.2.10]

> **Ex. 7.2.11:** Make two datasets based on the lists of columns below. Call the dataset with spatial columns data_geo and the other data.

```python
# Columns for `geo_data`, stored in `geo_columns`
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']

# Columns for `data`, stored in `select_columns`
hours = ['kl.{}-{}'.format(str(h).zfill(2), str(h+1).zfill(2)) for h in range(24)]
select_columns = ['road_name', 'date'] + hours
```

In [ ]:
# [Answer to Ex. 7.2.11]

> **Ex. 7.2.12:** Drop the duplicate rows in data_geo.

In [ ]:
# [Answer to Ex. 7.2.12]

**Formatting: wide and narrow format**

When talking about two-dimensional data (matrices, tables or dataframes, we can call it many things), we can either say that it is in wide or long format (see explanation here, "wide" and "long" are used interchangably). In Pandas we can use the commands stack and unstack to move between these formats.

The wide format has the advantage that it often requires less storage and is easier to read when printed. On the other hand the long format can be easier for modelling, because each observation has its own row. Turns out that the latter is what we most often need.

> **Ex. 7.2.13:** Turn the dataset from wide to long so hourly data is now vertically stacked. Store this dataset in a dataframe called data. Name the column with hourly information hour_period. 

> _Hint:_ pandas' melt function may be of use.

In [ ]:
# [Answer to Ex. 7.2.13]

**Categorical data**

Categorical data can contain Python objects, usually strings. These are smart if you have variables with string observations that are long and often repeated, e.g. with road names.

> **Ex. 7.2.14:** Use the `.astype` method to convert the type of the road_name column to categorical.

In [ ]:
# [Answer to Ex. 7.2.14]

### Structure temporal data

Pandas has native support for working with temporal data. This is handy as much 'big data' often has time stamps which we can make Pandas aware of. Once we have encoded temporal data it can be used to extract information such as the hour, second etc.

> **Ex. 7.2.15:** Create a new column called hour which contains the hour-of-day for each row. 

In [ ]:
# [Answer to Ex. 7.2.15]

>**Ex. 7.2.16:** Create a new column called time, that contains the time of the row in datetime format. Delete the old temporal columns (hour, hour_period, date) to save memory.

> _Hint:_ try making an intermediary series of strings that has all temporal information for the row; then use pandas to_datetime function where you can specify the format of the date string.

In [ ]:
# [Answer to Ex. 7.2.16]

> **Ex. 7.2.17:** Using your time column make a new column called weekday which stores the weekday (in values between 0 and 6) of the corresponding datetime.

> _Hint:_ try using the dt method for the series called time; dt has some relevant methods itself.

In [ ]:
# [Answer to Ex. 7.2.17]

### Statistical descriptions of traffic data

> **Ex. 7.2.18:** Print the "descriptive statistics" of the traffic column. Also show a kernel density estimate of the values.

> _Hint:_ Use the describe method of pandas dataframes for the first task. Use seaborn for the second. 

In [ ]:
# [Answer to Ex. 7.2.18]

> **Ex. 7.2.19:** Which road has the most average traffic?

In [ ]:
# [Answer to Ex. 7.2.19]

> **Ex. 7.2.20:** Compute annual, average road traffic during day hours (9-17). Which station had the least traffic in 2013? Which station has seen highest growth in traffic from 2013 to 2014?

In [58]:
# [Answer to Ex. 7.2.19]